In [1]:
import cv2
import torch
device = torch.device('cuda')

model_sala230_as - bias 0.15 , 
model_sala230_sw - bias 0.03

In [2]:
# import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('trt_steering_model_sala230_sw_100.pth')) # well trained road following model


model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('trt_collision_model_sala230_50.pth')) # well trained collision avoidance model

<All keys matched successfully>

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [4]:
import traitlets
from IPython.display import display
import ipywidgets
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_device=0, capture_fps = 10)

image = ipywidgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

In [ ]:
# camera.running = True

In [5]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [6]:
#Road Following sliders
speed_control_slider = ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.04, description='steering gain')
# steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
network_output_slider = ipywidgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})

# display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)
# display(speed_control_slider)

#Collision Avoidance sliders
blocked_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop') 
blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

# display(image)

# display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

In [7]:
import math

count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
x = 0.0
y = 0.0
speed_value = speed_control_slider.value


def execute(change):
    global angle, angle_last, blocked_slider, car, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_bias = steering_bias_slider.value
       
    image_preproc = preprocess(change['new']).to(device)
     
    #Collision Avoidance model:
    
    prob_blocked = float(F.softmax(model_trt_collision(image_preproc), dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
        else:
            #start of road following detection
            go_on = 1
            count_stops = 0
            xy = model_trt(image_preproc).detach().float().cpu().numpy().flatten()        
            x = float(xy[0])
            network_output_slider.value = x
            y = (0.5 - xy[1]) / 2.0
            speed_value = speed_control_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            x = 0.0 #set x steering to zero
            y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero (can set to turn as well)
        else:
            go_on = 1
            count_stops = 0
    
    car.steering = x *(-steer_gain) + steer_bias
    car.throttle = -speed_value 

execute({'new': camera.value})

In [8]:
camera.running = True
camera.observe(execute, names='value')

In [9]:
display(speed_control_slider, steering_gain_slider, steering_bias_slider)
display(image)
display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

FloatSlider(value=0.0, description='speed control', max=1.0, step=0.01)

FloatSlider(value=0.04, description='steering gain', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [10]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

# robot.stop()
car.throttle = 0.0

In [11]:
camera.running = False


In [ ]:
car.steering = 0